In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#define image and batch size
IMG_SIZE=224
BATCH_SIZE=32


In [6]:
#define data generator for train,validation and test sets
train_datagen=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [10]:
train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/dataset/Brain_Tumor_Detection/test',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

Found 2447 images belonging to 2 classes.
Found 611 images belonging to 2 classes.
Found 60 images belonging to 1 classes.


In [8]:
# Define the model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [9]:

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator,validation_data=val_generator,epochs=5)

model.save("Model.h5","label.txt")

Epoch 1/5
77/77 [==============================] - 527s 7s/step - loss: 0.4800 - accuracy: 0.8010 - val_loss: 0.2543 - val_accuracy: 0.8854
Epoch 2/5
77/77 [==============================] - 349s 5s/step - loss: 0.1926 - accuracy: 0.9297 - val_loss: 0.2481 - val_accuracy: 0.9067
Epoch 3/5
77/77 [==============================] - 386s 5s/step - loss: 0.1175 - accuracy: 0.9587 - val_loss: 0.0436 - val_accuracy: 0.9869
Epoch 4/5
77/77 [==============================] - 361s 5s/step - loss: 0.0445 - accuracy: 0.9853 - val_loss: 0.0534 - val_accuracy: 0.9836
Epoch 5/5
77/77 [==============================] - 340s 4s/step - loss: 0.0537 - accuracy: 0.9845 - val_loss: 0.0264 - val_accuracy: 0.9918


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [13]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
#load the save model
model=load_model('/content/drive/MyDrive/dataset/Model.h5')
#load and preprocess the test image
test_image_path='/content/drive/MyDrive/dataset/Brain_Tumor_Detection/train/yes/y0.jpg'
img=image.load_img(test_image_path,target_size=(224,224))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array,axis=0)#add batch dimension
img_array/=255.#normalize pixel values
#make predictions
prediction=model.predict(img_array)
#print the prediction
if prediction < 0.5:
  print("prediction:No tumor(probability:",prediction[0][0],")")
else:
   print("prediction: tumor present(probability:",prediction[0][0],")")

1/1 [==============================] - 0s 104ms/step
prediction: tumor present(probability: 1.0 )
